# Load Huggingface Pytorch Fill Mask Bert Model

In [1]:
import $ivy.`ai.djl:api:0.17.0`
import $ivy.`ai.djl.huggingface:tokenizers:0.17.0`
import $ivy.`ai.djl.pytorch:pytorch-model-zoo:0.17.0`
import $ivy.`ai.djl.pytorch:pytorch-engine:0.17.0`
import $ivy.`org.slf4j:slf4j-api:1.7.36`
import $ivy.`org.slf4j:slf4j-simple:1.7.36`
import $ivy.`com.lihaoyi::upickle:0.9.5`

Downloaded https://repo1.maven.org/maven2/com/lihaoyi/upickle_2.12/0.9.5/upickle_2.12-0.9.5.pom
Downloaded https://repo1.maven.org/maven2/com/lihaoyi/ujson_2.12/0.9.5/ujson_2.12-0.9.5.pom
Downloaded https://repo1.maven.org/maven2/com/lihaoyi/upack_2.12/0.9.5/upack_2.12-0.9.5.pom
Downloaded https://repo1.maven.org/maven2/com/lihaoyi/upickle-implicits_2.12/0.9.5/upickle-implicits_2.12-0.9.5.pom
Downloaded https://repo1.maven.org/maven2/com/lihaoyi/upickle-core_2.12/0.9.5/upickle-core_2.12-0.9.5.pom
Downloaded https://repo1.maven.org/maven2/com/lihaoyi/upack_2.12/0.9.5/upack_2.12-0.9.5-sources.jar
Downloaded https://repo1.maven.org/maven2/com/lihaoyi/upickle-core_2.12/0.9.5/upickle-core_2.12-0.9.5.jar
Downloaded https://repo1.maven.org/maven2/com/lihaoyi/ujson_2.12/0.9.5/ujson_2.12-0.9.5.jar
Downloaded https://repo1.maven.org/maven2/com/lihaoyi/upickle-core_2.12/0.9.5/upickle-core_2.12-0.9.5-sources.jar
Downloaded https://repo1.maven.org/maven2/com/lihaoyi/upickle-implicits_2.12/0.9.5/upi

import $ivy.$                  

import $ivy.$                                     

import $ivy.$                                        

import $ivy.$                                     

import $ivy.$                           

import $ivy.$                              

import $ivy.$                           

In [2]:
import java.io.IOException
import java.nio.file.{Files, Paths}
import java.util

import ai.djl.modality.Classifications
import ai.djl.modality.nlp.DefaultVocabulary
import ai.djl.modality.nlp.bert.BertTokenizer
import ai.djl.modality.nlp.Vocabulary
import ai.djl.ndarray.NDList
import ai.djl.repository.zoo.Criteria
import ai.djl.training.util.{DownloadUtils, ProgressBar}
import ai.djl.translate.{Batchifier, Translator, TranslatorContext}
import ai.djl.huggingface.tokenizers.HuggingFaceTokenizer

import java.io.IOException

import java.nio.file.{Files, Paths}

import java.util


import ai.djl.modality.Classifications

import ai.djl.modality.nlp.DefaultVocabulary

import ai.djl.modality.nlp.bert.BertTokenizer

import ai.djl.modality.nlp.Vocabulary

import ai.djl.ndarray.NDList

import ai.djl.repository.zoo.Criteria

import ai.djl.training.util.{DownloadUtils, ProgressBar}

import ai.djl.translate.{Batchifier, Translator, TranslatorContext}

import ai.djl.huggingface.tokenizers.HuggingFaceTokenizer

In [3]:
case class PredictedLabel(label: String, score: Double)

defined class PredictedLabel

In [4]:
import java.io.{IOException, InputStreamReader}
import java.net.URL
import java.nio.charset.StandardCharsets
import java.util

import ai.djl.util.JsonUtils
import com.google.gson.annotations.SerializedName

class VocabParser {
  @SerializedName("idx_to_token")
  var idx2token: util.Map[String, Long] = _

}

object VocabParser {
  def parseToken(file: URL) = {
    try {
      val is = file.openStream()
      val reader = new InputStreamReader(is, StandardCharsets.UTF_8)
      JsonUtils.GSON.fromJson(reader, classOf[VocabParser]).idx2token
    } catch {
      case e: IOException => throw new IllegalArgumentException("Invalid url: " + file, e)
    }
  }
}

import java.io.{IOException, InputStreamReader}

import java.net.URL

import java.nio.charset.StandardCharsets

import java.util


import ai.djl.util.JsonUtils

import com.google.gson.annotations.SerializedName


defined class VocabParser
defined object VocabParser

In [4]:
import java.net.URL

val url = Paths.get("../build/huggingface/zero-shot-classification/pytorch/bart-large-mnli/vocab.json").toUri.toURL

import java.net.URL


url: URL = file:/Users/donglin/Workspace/deep-learning-scala/notebook/../build/huggingface/zero-shot-classification/pytorch/bart-large-mnli/vocab.json

In [15]:
import ai.djl.util.JsonUtils

import ai.djl.util.JsonUtils

In [16]:
import java.io.{IOException, InputStreamReader}
import java.nio.charset.StandardCharsets

val is = url.openStream()
val reader = new InputStreamReader(is, StandardCharsets.UTF_8)

import java.io.{IOException, InputStreamReader}

import java.nio.charset.StandardCharsets


is: java.io.InputStream = java.io.BufferedInputStream@695cbea8
reader: InputStreamReader = java.io.InputStreamReader@79610fff

In [18]:
JsonUtils.GSON.fromJson(reader, classOf[util.Map[String, Long]])

: 

In [5]:
val jsonStr = scala.io.Source.fromFile(url.getPath).mkString

jsonStr: String = "{\"<s>\":0,\"<pad>\":1,\"</s>\":2,\"<unk>\":3,\".\":4,\"\u0120the\":5,\",\":6,\"\u0120to\":7,\"\u0120and\":8,\"\u0120of\":9,\"\u0120a\":10,\"\u0120in\":11,\"-\":12,\"\u0120for\":13,\"\u0120that\":14,\"\u0120on\":15,\"\u0120is\":16,\"\u00e2\u0122\":17,\"'s\":18,\"\u0120with\":19,\"\u0120The\":20,\"\u0120was\":21,\"\u0120\\\"\":22,\"\u0120at\":23,\"\u0120it\":24,\"\u0120as\":25,\"\u0120said\":26,\"\u013b\":27,\"\u0120be\":28,\"s\":29,\"\u0120by\":30,\"\u0120from\":31,\"\u0120are\":32,\"\u0120have\":33,\"\u0120has\":34,\":\":35,\"\u0120(\":36,\"\u0120he\":37,\"\u0120I\":38,\"\u0120his\":39,\"\u0120will\":40,\"\u0120an\":41,\"\u0120this\":42,\")\":43,\"\u0120\u00e2\u0122\":44,\"\u0120not\":45,\"\u013f\":46,\"\u0120you\":47,\"\u013e\":48,\"\u0120their\":49,\"\u0120or\":50,\"\u0120they\":51,\"\u0120we\":52,\"\u0120but\":53,\"\u0120who\":54,\"\u0120more\":55,\"\u0120had\":56,\"\u0120been\":57,\"\u0120were\":58,\"\u0120about\":59,\",\\\"\":60,\"\u0120which\":61,\"\u0120up\":

In [14]:
val vocab = ujson.read(jsonStr).obj.keySet.toSeq

vocab: Seq[String] = Vector(
  "<s>",
  "<pad>",
  "</s>",
  "<unk>",
  ".",
  "\u0120the",
  ",",
  "\u0120to",
  "\u0120and",
  "\u0120of",
  "\u0120a",
  "\u0120in",
  "-",
  "\u0120for",
  "\u0120that",
  "\u0120on",
  "\u0120is",
  "\u00e2\u0122",
  "'s",
  "\u0120with",
  "\u0120The",
  "\u0120was",
  "\u0120\"",
  "\u0120at",
  "\u0120it",
  "\u0120as",
  "\u0120said",
  "\u013b",
  "\u0120be",
  "s",
  "\u0120by",
  "\u0120from",
  "\u0120are",
  "\u0120have",
  "\u0120has",
  ":",
  "\u0120(",
  "\u0120he",
...

In [12]:
data.obj.map {case (key, value) => }

res11: collection.Set[String] = Set(
  "<s>",
  "<pad>",
  "</s>",
  "<unk>",
  ".",
  "\u0120the",
  ",",
  "\u0120to",
  "\u0120and",
  "\u0120of",
  "\u0120a",
  "\u0120in",
  "-",
  "\u0120for",
  "\u0120that",
  "\u0120on",
  "\u0120is",
  "\u00e2\u0122",
  "'s",
  "\u0120with",
  "\u0120The",
  "\u0120was",
  "\u0120\"",
  "\u0120at",
  "\u0120it",
  "\u0120as",
  "\u0120said",
  "\u013b",
  "\u0120be",
  "s",
  "\u0120by",
  "\u0120from",
  "\u0120are",
  "\u0120have",
  "\u0120has",
  ":",
  "\u0120(",
  "\u0120he",
...

In [7]:
val path = Paths.get("/Users/donglin/Workspace/deep-learning-scala/build/huggingface/zero-shot-classification/pytorch/bart-large-mnli/vocab.json").toUri.toURL
// val vocabulary = DefaultVocabulary.builder
//   .optMinFrequency(1)
//   .addFromCustomizedFile(path, VocabParser.parseToken)
//   .optUnknownToken("[UNK]")
//   .build

: 

In [4]:
class HFZeroShotClsTranslator extends Translator[String, Seq[PredictedLabel]] {

  private var vocabulary: DefaultVocabulary = _
  private var tokenizer: HuggingFaceTokenizer = _
  private var tokenList: Array[String] = _
  private final val MaskToken = "[MASK]"
  private final val TopK = 5

  override def prepare(ctx: TranslatorContext): Unit = {
    val path = Paths.get("../build/huggingface/fill_mask/pytorch/bert-base-uncased/vocab.txt")
    vocabulary = DefaultVocabulary.builder
      .optMinFrequency(1)
      .addFromTextFile(path)
      .optUnknownToken("[UNK]")
      .build
    tokenizer = HuggingFaceTokenizer.newInstance("bert-base-uncased")
  }

  override def processInput(ctx: TranslatorContext, input: String): NDList = {
    val token = tokenizer.encode(input.toLowerCase().replace(MaskToken.toLowerCase(), MaskToken))
    // get the encoded tokens that would be used in precessOutput
    tokenList = token.getTokens
    // map the tokens(String) to indices(long)

    val manager = ctx.getNDManager
    val indices = tokenList.map(vocabulary.getIndex)
    val attentionMask = token.getAttentionMask.map(i => i)
    val indicesArray = manager.create(indices)
    val attentionMaskArray = manager.create(attentionMask)

    new NDList(indicesArray, attentionMaskArray)
  }

  override def processOutput(ctx: TranslatorContext, list: NDList): Seq[PredictedToken] = {
    val maskIndex = tokenList.zipWithIndex.find(_._1 == MaskToken).map(_._2).getOrElse(-1)
    if (maskIndex == -1) {
      Seq.empty[PredictedToken]
    } else {
      val ndArray = list.get(0)
      val shape = ndArray.getShape
      val len = shape.get(1)

      (1 to TopK).map { i =>
        val out = ndArray.get(maskIndex).argSort().getLong(len - i)
        PredictedToken(vocabulary.getToken(out), ndArray.getFloat(maskIndex, out))
      }
    }
  }

  override def getBatchifier: Batchifier = Batchifier.STACK
}

defined class HFBertFillMaskTranslator

In [5]:
val input = "Paris is the [MASK] of France.".toLowerCase.replace("[mask]", "[MASK]")

val translator = new HFBertFillMaskTranslator()
val criteria = Criteria.builder
  .setTypes(classOf[String], classOf[Seq[PredictedToken]])
  .optModelPath(Paths.get("../build/huggingface/fill_mask/pytorch/bert-base-uncased/"))
  .optTranslator(translator)
  .optProgress(new ProgressBar)
  .build

val model = criteria.loadModel()

val predictor = model.newPredictor(translator)

val predictResult = predictor.predict(input)
predictResult.foreach(println(_))

Loading:     100% |████████████████████████████████████████|


[scala-interpreter-1] INFO ai.djl.pytorch.engine.PtEngine - Number of inter-op threads is 4
[scala-interpreter-1] INFO ai.djl.pytorch.engine.PtEngine - Number of intra-op threads is 4
[scala-interpreter-1] INFO ai.djl.huggingface.tokenizers.jni.LibUtils - Extracting native/lib/osx-x86_64/libtokenizers.dylib to cache ...


PredictedToken(capital,18.19974136352539)
PredictedToken(heart,10.769936561584473)
PredictedToken(center,10.469231605529785)
PredictedToken(centre,10.209856986999512)
PredictedToken(city,9.985564231872559)


input: String = "paris is the [MASK] of france."
translator: HFBertFillMaskTranslator = ammonite.$sess.cmd3$Helper$HFBertFillMaskTranslator@241526f0
criteria: Criteria[String, Seq[PredictedToken]] = Criteria:
	Application: UNDEFINED
	Input: class java.lang.String
	Output: interface scala.collection.Seq
	ModelZoo: ai.djl.localmodelzoo

model: ai.djl.repository.zoo.ZooModel[String, Seq[PredictedToken]] = ai.djl.repository.zoo.ZooModel@6b002e0e
predictor: ai.djl.inference.Predictor[String, Seq[PredictedToken]] = ai.djl.inference.Predictor@c132fed
predictResult: Seq[PredictedToken] = Vector(
  PredictedToken("capital", 18.19974136352539),
  PredictedToken("heart", 10.769936561584473),
  PredictedToken("center", 10.469231605529785),
  PredictedToken("centre", 10.209856986999512),
  PredictedToken("city", 9.985564231872559)
)